In [1]:
import requests


## Population APIs
Total Population: https://data.worldbank.org/indicator/SP.POP.TOTL

In [39]:
# get the World Bank 
total_population_data = []
for page in range(17):
    payload = {'format': 'json', 'per_page': '1000', 'date':'1960:2022', 'page':page}
    population_json = requests.get('http://api.worldbank.org/v2/countries/indicators/SP.POP.TOTL', params=payload)
    total_population_data+=population_json.json()


## Land Development APIs
1. Forest: https://data.worldbank.org/indicator/AG.LND.FRST.ZS?view=map&year=2020
2. Permanent cropland: https://data.worldbank.org/indicator/AG.LND.CROP.ZS?view=chart
3. Agricultural Land: https://data.worldbank.org/indicator/AG.LND.AGRI.ZS?end=2018&start=1961&view=chart
4. Arable Land: https://data.worldbank.org/indicator/AG.LND.ARBL.ZS?end=2018&start=1961&view=chart
5. Land Cereal production : https://data.worldbank.org/indicator/AG.LND.CREL.HA?view=chart


## Living Condition APIs
1. Rural population: https://data.worldbank.org/indicator/SP.RUR.TOTL.ZS?view=chart
2. Urban Population: https://data.worldbank.org/indicator/SP.URB.TOTL.IN.ZS?view=chart
3. Male Employment:  https://data.worldbank.org/indicator/SL.AGR.EMPL.MA.ZS?view=chart
4. Female Employement: https://data.worldbank.org/indicator/SL.AGR.EMPL.FE.ZS?view=chart



## Agricultural Practice APIs
1. Fertilizer Use: https://data.worldbank.org/indicator/AG.CON.FERT.ZS?view=chart
2. Cereal Production: https://data.worldbank.org/indicator/AG.YLD.CREL.KG?view=chart
3. % of GDP value: https://data.worldbank.org/indicator/NV.AGR.TOTL.ZS?end=2021&start=1960&view=chart

## World Issue APIs
1. Under 5 mortality: https://data.worldbank.org/indicator/SH.DYN.MORT?view=chart
2. Greenhouse Gases: https://data.worldbank.org/indicator/EN.ATM.GHGT.KT.CE?view=chart
3. CO2 emissions: https://data.worldbank.org/indicator/EN.ATM.CO2E.KT?view=chart
4. Poverty : https://data.worldbank.org/indicator/SI.POV.DDAY?view=chart
